In [1]:
import pandas as pd

import ast

In [3]:
foodRaw_df = pd.read_csv('../../Resources/CSVs/all_menu_items.csv')

foodRaw_df.tail()

,Restaurant,Food_Name,Nutrients,Calories
30,Saladworks,Buffalo Chicken Panini,"[{'value': 43, 'attr_id': 203}, {'value': 29, ...",660
31,Saladworks,Chicken Caesar,"[{'value': 42, 'attr_id': 203}, {'value': 4, '...",290
32,Saladworks,Cobb,"[{'value': 23, 'attr_id': 203}, {'value': 16, ...",270
33,Saladworks,Turkey Melt,"[{'value': 47, 'attr_id': 203}, {'value': 40, ...",800
34,Saladworks,Mandarin Chicken,"[{'value': 12, 'attr_id': 203}, {'value': 7, '...",230


In [4]:
nutrient_map = pd.read_csv('../../Resources/CSVs/Nutritionix API v2 - Full Nutrient USDA Field Mapping.csv')

nutrient_map = nutrient_map.set_index('attr_id')

nutrient_map.head()

,2018 NFP,usda_tag,name,unit,notes,bulk_csv_field,cxh_attribute_name,Unnamed: 8
attr_id,,,,,,,,
203,1.0,PROCNT,Protein,g,NaN,nf_protein,PRO-,NaN
204,1.0,FAT,Total lipid (fat),g,NaN,nf_total_fat,FAT,NaN
205,1.0,CHOCDF,"Carbohydrate, by difference",g,NaN,nf_total_carbohydrate,CHO-,NaN
207,0.0,ASH,Ash,g,NaN,NaN,NaN,NaN
208,1.0,ENERC_KCAL,Energy,kcal,NaN,nf_calories,ENER-,NaN


In [6]:

#For Loop to iterate through the API raw data.
for index,row in foodRaw_df.iterrows(): 
    
    nutrients = ast.literal_eval(row['Nutrients']) #The original column of Nutrients looked like a list of dictionaries but in reality,  was a string. 
    #ast.literal_eval -->interprets "strings, bytes, numbers, tuples, lists, dicts, sets, booleans, and None." and returns it in the right format --> anything else is an error.
    
    #Nested For Loop to convert each of the nutrients for each menu item.
    for nutrient in nutrients: 
        
        #locates the name of the attr_id in the nutrient_map csv. 
        name = nutrient_map.loc[nutrient['attr_id'],'name'] 

        #Unit Conversion.
        conversion_dict = { 
            'Protein' : 4, #To cal
            'Carbohydrate, by difference' : 4, #to cal
            'Total lipid (fat)' : 9, #to cal
            'Vitamin A, IU' : 0.3, #to mcg RAE
            'Fatty acids, total trans' : 10, #to cal
            'Fatty acids, total saturated' : 9, #to cal
            'Sugars, total' : 4} #to cal.
        
        #Dictionary to apply unit conversion factor.
        if name in conversion_dict.keys():
            conversionFactor = conversion_dict[name]
        else:
            conversionFactor = 1
        
        value = nutrient['value'] 
        
       #If nurtient is in the columns.
        if name in foodRaw_df.columns:

            #Apply conversion Factor.
            foodRaw_df.loc[index,name] = value * conversionFactor 
        
        #If nutrient is not in the columns.
        else: 
            foodRaw_df[name] ='' #Create empty column.
            foodRaw_df.loc[index,name] = value * conversionFactor #Apply conversion factor.
        
food_df = foodRaw_df.replace(r"^\s*$", 0, regex = True) #Finds and replace empty cells with 0. empty cell pattern = r"^\s*$"
food_df = food_df.drop(columns=["Nutrients"])
food_df.head()

,Restaurant,Food_Name,Calories,Protein,Total lipid (fat),"Carbohydrate, by difference",Energy,"Sugars, total","Fiber, total dietary","Sodium, Na",Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated","Calcium, Ca","Iron, Fe","Vitamin A, IU","Vitamin C, total ascorbic acid"
0,Panera Bread,"Caesar Salad with Chicken, Whole",450,140,243.0,68,450,8,2,770,120,0,63.0,0,0.0,0.0,0
1,Panera Bread,"Mediterranean Veggie Sandwich on Tomato Basil,...",440,72,117.0,268,440,32,6,1230,10,0,27.0,0,0.0,0.0,0
2,Panera Bread,Napa Almond Chicken Salad Sandwich on Country ...,630,112,288.0,240,630,48,5,680,70,0,40.5,0,0.0,0.0,0
3,Panera Bread,"Fuji Apple Salad with Chicken, Whole",570,132,315.0,144,570,80,6,590,95,0,63.0,0,0.0,0.0,0
4,Panera Bread,"Modern Greek Salad with Quinoa, Whole",530,48,387.0,108,530,16,7,870,15,0,63.0,0,0.0,0.0,0


In [7]:
food_df.to_csv("projectETL_restaurant.csv", index=False, header=True)

In [5]:
#Organizes the order of the columns. 

foodFiltered_df = food_df[['Restaurant',
                           'Food_Name',
                           'Calories',
                           'Protein',
                           'Carbohydrate, by difference',
                           'Fiber, total dietary',
                           'Sugars, total',
                           'Total lipid (fat)',
                           'Fatty acids, total trans',
                           'Fatty acids, total saturated',
                           'Cholesterol',
                           'Calcium, Ca',
                           'Iron, Fe',
                           'Sodium, Na',
                           'Vitamin A, IU',
                          'Vitamin C, total ascorbic acid']]

#Parameters to rename.
columns_name = {
    'Protein' : 'Protein (Cal)',
    'Carbohydrate, by difference' : 'Carbohydrate (Cal)',
    'Fiber, total dietary' : 'Fiber (gr)',
    'Sugars, total' : 'Total Sugar (Cal)',
    'Total lipid (fat)' : 'Total Fat (Cal)',
    'Cholesterol' : 'Cholesterol (mg)',
    'Calcium, Ca' : 'Calcium (mg)',
    'Iron, Fe' : 'Iron (mg)',
    'Sodium, Na' : 'Sodium (mg)',
    'Vitamin A, IU' : 'Vitamin A (mcg RAE)',
    'Vitamin C, total ascorbic acid' : 'Vitamin C (mg)',
    'Fatty acids, total trans' : 'Total Trans Fat (Cal)',
    'Fatty acids, total saturated' : 'Total Saturated Fat (Cal)'
    
}

foodFiltered_df = foodFiltered_df.rename(columns = columns_name)

foodFiltered_df.head()


,Restaurant,Food_Name,Calories,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (Cal),Total Fat (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),Sodium (mg),Vitamin A (mcg RAE),Vitamin C (mg)
0,Panera Bread,"Caesar Salad with Chicken, Whole",450,140,68,2,8,243.0,0,63.0,120,0,0.0,770,0.0,0
1,Panera Bread,"Mediterranean Veggie Sandwich on Tomato Basil,...",440,72,268,6,32,117.0,0,27.0,10,0,0.0,1230,0.0,0
2,Panera Bread,Napa Almond Chicken Salad Sandwich on Country ...,630,112,240,5,48,288.0,0,40.5,70,0,0.0,680,0.0,0
3,Panera Bread,"Fuji Apple Salad with Chicken, Whole",570,132,144,6,80,315.0,0,63.0,95,0,0.0,590,0.0,0
4,Panera Bread,"Modern Greek Salad with Quinoa, Whole",530,48,108,7,16,387.0,0,63.0,15,0,0.0,870,0.0,0


In [6]:
#Calorie intake per gender, life style and age.

calorieNeeds_male = pd.read_excel('../../FDA Tables XLSX/Appendix 2. Estimated Calorie Needs -2020 Dietary Guidelines _ health.gov_0002.xlsx', sheet_name = 'Males')

#Removes ages less than 14.
calorieNeeds_male = calorieNeeds_male[calorieNeeds_male['AGE'] >= 14]

calorieNeeds_male = calorieNeeds_male.reset_index(drop= True)

calorieNeeds_male.head()

,AGE,Sedentary,Moderately active,Active
0,14,2000,2400,2800
1,15,2200,2600,3000
2,16,2400,2800,3200
3,17,2400,2800,3200
4,18,2400,2800,3200


In [7]:
#Male nutritional parameters per meal. Source: FDA 

nutritionParameters_dict = {'Protein (Cal)' : ((12 + 3) / 2)/100, # % kCal
                           'Carbohydrate (Cal)' : ((10 + 25) / 2) / 100, # % kCcal
                           'Fiber (gr)' : (8 + 30) / 2, 
                           'Total Sugar (Cal)': .04, # % kCal
                           'Total lipid (Cal)': ((12 + 8) / 2)/ 100, # % kCal
                           'Total Trans Fat (Cal)': .01, # % kCal
                           'Total Saturated Fat (Cal)': 0.03, # % kCal
                           'Cholesterol (mg)' : 100, 
                           'Calcium (mg)': (835 + 330) / 2,
                           'Iron (mg)': (5 + 15) / 2,
                           'Sodium, (mg)': (770 + 150)/2,
                           'Vitamin A (mcg RAE)': (1000 + 230) / 2,
                            'Vitamin C (mg)' : 21,
                            'Calories per Meal' : 1/3}
    


In [8]:
#Code to combine FDA calorie intake and the recomended nutritonal parameters.

#Empty list to store DataFrames.
maleInfo = []

#For loop to iterate through each lifestyle in the FDA calorie intake recomendation DataFrame
for lifeStyle in ['Sedentary','Moderately active','Active']:
    
    #Temporary DataFrame to make calculations based on the age column and lifestyle.
    lifeStyle_df = calorieNeeds_male[['AGE', lifeStyle]]
    
    #For loop to iterate through the temporary DataFrame.
    for index,row in lifeStyle_df.iterrows():

        #For loop to appy each of the FDA nutritional parameters to each of the rows in the temporary lifestyle calorie intake DataFrame.
        for key in nutritionParameters_dict.keys():

            #If the key is NOT in the columns of the DataFrame.
            if key not in lifeStyle_df.columns:

                #Creates an empty column using the keys of the FDA nutrion dictionary.
                lifeStyle_df[key] = ''

                #Some FDA parameters are % kCal. These parameters need to multiply dictionary value and calorie intake.
                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)
                
                #The rest of the FDA parameters are recomandations based on grams.
                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)

            #If the key is in the columns of the DataFrame.
            else:

                #Same operation as before.
                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)

                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)

    #Creates a list with the title of the columns in the temporary DataFrame.
    columns = lifeStyle_df.columns.to_list()

    #Converts all the columns to numeric.
    lifeStyle_df[columns] = lifeStyle_df[columns].apply(pd.to_numeric, axis=1) 

    #Groups by carolie intake, averages all the values, resets the index and drops the column AGE. 
    lifeStyle_df = lifeStyle_df.groupby(lifeStyle).mean().reset_index().drop(['AGE'], axis=1)

    #Appends the temporary DataFrame to the list of DataFrames.                 
    maleInfo.append(lifeStyle_df)
                        
maleInfo[0] #List of DataFrames for males and the intake limits for each meal. 

,Sedentary,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (Cal),Total lipid (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),"Sodium, (mg)",Vitamin A (mcg RAE),Vitamin C (mg),Calories per Meal
0,2000.0,150.0,350.0,19.0,80.0,200.0,20.0,60.0,100.0,582.5,10.0,460.0,615.0,21.0,666.67
1,2200.0,165.0,385.0,19.0,88.0,220.0,22.0,66.0,100.0,582.5,10.0,460.0,615.0,21.0,733.33
2,2400.0,180.0,420.0,19.0,96.0,240.0,24.0,72.0,100.0,582.5,10.0,460.0,615.0,21.0,800.00
3,2600.0,195.0,455.0,19.0,104.0,260.0,26.0,78.0,100.0,582.5,10.0,460.0,615.0,21.0,866.67


elif row['AGE'] > 19 & row['AGE'] <= 30 :

            for key in maleNutrition19to30_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition19to30_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition19to30_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in maleNutrition31to50_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition31to50_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition31to50_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in maleNutrition51_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition51_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition51_dict[key],2)

In [9]:
#Load kcal requirments from FDA

calorieNeeds_female = pd.read_excel('../../FDA Tables XLSX/Appendix 2. Estimated Calorie Needs -2020 Dietary Guidelines _ health.gov_0002.xlsx', sheet_name = 'Females')

calorieNeeds_female = calorieNeeds_female[calorieNeeds_female['AGE'] >= 14]

calorieNeeds_female =  calorieNeeds_female.reset_index(drop= True)

calorieNeeds_female.head()

,AGE,Sedentary,Moderately active,Active
0,14,1800,2000,2400
1,15,1800,2000,2400
2,16,1800,2000,2400
3,17,1800,2000,2400
4,18,1800,2000,2400


In [10]:
femaleInfo = []

for lifeStyle in ['Sedentary','Moderately active','Active']:
    
    lifeStyle_df = calorieNeeds_female[['AGE', lifeStyle]]
    
    for index,row in lifeStyle_df.iterrows():

        for key in nutritionParameters_dict.keys():

            if key not in lifeStyle_df.columns:

                lifeStyle_df[key] = ''

                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)

                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)

            else:

                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)

                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)
                    
    columns = lifeStyle_df.columns.to_list()

    lifeStyle_df[columns] = lifeStyle_df[columns].apply(pd.to_numeric, axis=1) 

    lifeStyle_df = lifeStyle_df.groupby(lifeStyle).mean().reset_index().drop(['AGE'], axis=1)
       
                        
    femaleInfo.append(lifeStyle_df)
                        
femaleInfo[0] #List of DataFrames for males and the intake limits for each meal. 

,Sedentary,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (Cal),Total lipid (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),"Sodium, (mg)",Vitamin A (mcg RAE),Vitamin C (mg),Calories per Meal
0,1600.0,120.0,280.0,19.0,64.0,160.0,16.0,48.0,100.0,582.5,10.0,460.0,615.0,21.0,533.33
1,1800.0,135.0,315.0,19.0,72.0,180.0,18.0,54.0,100.0,582.5,10.0,460.0,615.0,21.0,600.00
2,2000.0,150.0,350.0,19.0,80.0,200.0,20.0,60.0,100.0,582.5,10.0,460.0,615.0,21.0,666.67


In [11]:
#Stores the DataFrames in the system.

%store maleInfo

%store femaleInfo

%store foodFiltered_df

Stored 'maleInfo' (list)
Stored 'femaleInfo' (list)
Stored 'foodFiltered_df' (DataFrame)
